In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import zipfile,os 

In [ ]:
base_dir  = '/content/drive/MyDrive/dataset-5000'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'val')

In [ ]:
os.listdir('/content/drive/MyDrive/dataset-5000/train')
os.listdir('/content/drive/MyDrive/dataset-5000/val')

['nonhijab', 'hijab']

#torch

##Model old

In [ ]:
import torch
import torch.nn as nn

class CustomVGG(nn.Module):
    def __init__(self, num_classes=2):
        super(CustomVGG, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3,padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256 ,256,kernel_size = 3,padding = 1 ),
           
        )
        self.avgpool = nn.AdaptiveAvgPool2d((7*7*512))
        self.classifier =nn.Sequential(
            
           torch.nn.Linear(in_features=(7*7*512), out_features=num_classes,bias=True)
           
           
           
        )

    def forward(self,x):
        
        x=self.features(x)
        
        x=self.avgpool(x)
        
        x=torch.flatten(x,start_dim = 1)
        
        x=self.classifier(x)
        
        
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')




In [ ]:
import torch
import torch.nn as nn
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class CustomVGG(nn.Module):
    def __init__(self, num_classes=2):
        super().__init__()
        self.network = nn.Sequential(
            
            nn.Conv2d(3, 32, kernel_size = 3, padding = 1),
            nn.ReLU(),
            nn.Conv2d(32,64, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
        
            nn.Conv2d(64, 128, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.Conv2d(128 ,128, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            
            nn.Conv2d(128, 256, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.Conv2d(256,256, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            
            nn.Flatten(),
            nn.Linear(82944,1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512,num_classes)
        )
    
    def forward(self, xb):
        return self.network(xb)

In [ ]:
import torch
import torch.nn as nn
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import torch.nn.functional as F

class CustomVGG(nn.Module):

    # Defining the Constructor
    def __init__(self, num_classes=3):
        super(CustomVGG, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=12, kernel_size=3, stride=1, padding=1)
        
        self.conv2 = nn.Conv2d(in_channels=12, out_channels=24, kernel_size=3, stride=1, padding=1)
        
        # We in the end apply max pooling with a kernel size of 2
        self.pool = nn.MaxPool2d(kernel_size=2)
        
        # A drop layer deletes 20% of the features to help prevent overfitting
        self.drop = nn.Dropout2d(p=0.2)
        
        # Our 128x128 image tensors will be pooled twice with a kernel size of 2. 128/2/2 is 32.
        # This means that our feature tensors are now 32 x 32, and we've generated 24 of them
        
        # We need to flatten these in order to feed them to a fully-connected layer
        self.fc = nn.Linear(in_features=32 * 32 * 24, out_features=num_classes)

    def forward(self, x):
        # In the forward function, pass the data through the layers we defined in the init function
        
        # Use a ReLU activation function after layer 1 (convolution 1 and pool)
        x = F.relu(self.pool(self.conv1(x))) 
        
        # Use a ReLU activation function after layer 2
        x = F.relu(self.pool(self.conv2(x)))  
        
        # Select some features to drop to prevent overfitting (only drop during training)
        x = F.dropout(self.drop(x), training=self.training)
        
        # Flatten
        x = x.view(-1, 32 * 32 * 24)
        # Feed to fully-connected layer to predict class
        x = self.fc(x)
        # Return class probabilities via a log_softmax function 
        return torch.log_softmax(x, dim=1)

##last model

In [ ]:
import torch
import torch.nn as nn
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import torch.nn.functional as F

class CustomVGG(nn.Module):
    def __init__(self):
        super().__init__()

        # onvolutional layers (3,16,32)
        self.conv1 = nn.Conv2d(in_channels = 3, out_channels = 16, kernel_size=(5, 5), stride=2, padding=1)
        self.conv2 = nn.Conv2d(in_channels = 16, out_channels = 32, kernel_size=(5, 5), stride=2, padding=1)
        self.conv3 = nn.Conv2d(in_channels = 32, out_channels = 64, kernel_size=(3, 3), padding=1)

        # conected layers
        self.fc1 = nn.Linear(in_features= 64 * 6 * 6, out_features=500)
        self.fc2 = nn.Linear(in_features=500, out_features=50)
        self.fc3 = nn.Linear(in_features=50, out_features=2)


    def forward(self, X):

        X = F.relu(self.conv1(X))
        X = F.max_pool2d(X, 2)

        X = F.relu(self.conv2(X))
        X = F.max_pool2d(X, 2)

        X = F.relu(self.conv3(X))
        X = F.max_pool2d(X, 2)

        X = X.view(X.shape[0], -1)
        X = F.relu(self.fc1(X))
        X = F.relu(self.fc2(X))
        X = self.fc3(X)

        return X

##Data

In [ ]:
# import os
# os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:512"

In [ ]:
# import torch
# torch.cuda.empty_cache()
# import gc
# try:
#    del model
# except:
#   pass
# gc.collect()

In [ ]:
model = CustomVGG().to(device)

In [ ]:
import torch
from torchvision import datasets, transforms

# Define data transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load train and test datasets
train_dataset = datasets.ImageFolder('/content/drive/MyDrive/dataset-5000/train', transform=transform)
test_dataset = datasets.ImageFolder('/content/drive/MyDrive/dataset-5000/val', transform=transform)

# Create data loaders
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64)

##Train

In [ ]:
# Define your loss function and optimizer
criterion = torch.nn.CrossEntropyLoss()
#optimizer = torch.optim.SGD(model.parameters(), lr=0.001)
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)

num_epochs = 15

def train_test():
    # Loop over your data for multiple epochs
    for epoch in range(num_epochs):
        for i, (images_batch, labels_batch )in enumerate(train_loader):
            images_batch, labels_batch = images_batch.cuda(), labels_batch.cuda() 
            # Zero out gradients from previous iteration
            optimizer.zero_grad()
            
            # Forward pass through the network
            output_batch = model(images_batch)
            
            # Compute loss between predicted and true labels
            loss = criterion(output_batch , labels_batch)
            
            # Backpropagate gradients through the network
            loss.backward()
            
            # Update weights using optimizer step
            optimizer.step()
            # Print the loss for this step
            #print('Loss: {:.6f}'.format(loss.item()))
            # Compute accuracy
            _, predicted = torch.max(output_batch.data, 1)
            correct = (predicted == labels_batch).sum().item()
            total = labels_batch.size(0)
            accuracy = correct / total

            # Print loss and accuracy
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'
                  .format(epoch+1, num_epochs, i+1, len(train_loader), loss.item(), accuracy*100))

    # Evaluate your model on test data after training is complete
    torch.save(model.state_dict(), 'hijab_model_torch.pth')

    with torch.no_grad():
        correct = 0
        
        for images_batch , labels_batch in test_loader:
            images_batch, labels_batch = images_batch.cuda(), labels_batch.cuda() 
            output=model(images_batch)
            
            _,predicted=torch.max(output.data,dim=-1)
            
            correct+= (predicted == labels_batch).sum().item()

        accuracy=(correct/len(test_loader.dataset))*100
        print(f"Accuracy: {accuracy}%")

In [ ]:
train_test() 

Epoch [1/15], Step [1/126], Loss: 0.6907, Accuracy: 54.69%
Epoch [1/15], Step [2/126], Loss: 0.6385, Accuracy: 64.06%
Epoch [1/15], Step [3/126], Loss: 0.8245, Accuracy: 54.69%
Epoch [1/15], Step [4/126], Loss: 0.6847, Accuracy: 53.12%
Epoch [1/15], Step [5/126], Loss: 0.6758, Accuracy: 50.00%
Epoch [1/15], Step [6/126], Loss: 0.6801, Accuracy: 59.38%
Epoch [1/15], Step [7/126], Loss: 0.6728, Accuracy: 76.56%
Epoch [1/15], Step [8/126], Loss: 0.6749, Accuracy: 62.50%
Epoch [1/15], Step [9/126], Loss: 0.6618, Accuracy: 59.38%
Epoch [1/15], Step [10/126], Loss: 0.6689, Accuracy: 53.12%
Epoch [1/15], Step [11/126], Loss: 0.6460, Accuracy: 59.38%
Epoch [1/15], Step [12/126], Loss: 0.6025, Accuracy: 68.75%
Epoch [1/15], Step [13/126], Loss: 0.5839, Accuracy: 71.88%
Epoch [1/15], Step [14/126], Loss: 0.5967, Accuracy: 68.75%
Epoch [1/15], Step [15/126], Loss: 0.5657, Accuracy: 75.00%
Epoch [1/15], Step [16/126], Loss: 0.5385, Accuracy: 81.25%
Epoch [1/15], Step [17/126], Loss: 0.4973, Accura

RuntimeError: ignored

In [ ]:
import torch
from PIL import Image
import cv2

# load the saved model
model = CustomVGG()
model.load_state_dict(torch.load('hijab_model_torch.pth')) 
model.to(device)

CustomVGG(
  (conv1): Conv2d(3, 16, kernel_size=(5, 5), stride=(2, 2), padding=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(5, 5), stride=(2, 2), padding=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=2304, out_features=500, bias=True)
  (fc2): Linear(in_features=500, out_features=50, bias=True)
  (fc3): Linear(in_features=50, out_features=2, bias=True)
)

#test

In [ ]:
with torch.no_grad():
        correct = 0
        for images_batch , labels_batch in test_loader:
            images_batch, labels_batch = images_batch.cuda(), labels_batch.cuda() 
            output=model(images_batch)
            
            _,predicted=torch.max(output.data,dim=-1)
            
            correct+= (predicted == labels_batch).sum().item()

        accuracy=(correct/len(test_loader.dataset))*100
        print(f"Accuracy: {accuracy}%")

Accuracy: 99.6003996003996%


In [ ]:

transform_normjj = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

classes = train_loader.dataset.classes #['hijab', 'nonhijab']
def pred_glasses(img):
         img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
         # get normalized image
         img_normalized = transform_normjj(img).float()
         img_normalized = img_normalized.unsqueeze_(0)
         img_normalized = img_normalized.to(device)
         with torch.no_grad():
            model.eval()  
            output = model(img_normalized)
            index  = output.data.cpu().numpy().argmax()
            class_name = classes[index]
            return class_name, index

In [ ]:
img = cv2.imread('/content/324659816_1167662230544924_8200639109336242480_n.jpg')
pred_glasses(img) 

('nonhijab', 1)

In [ ]:
!wget https://m.media-amazon.com/images/I/41+x3N0H14L._AC_UF1000,1000_QL80_.jpg

In [ ]:
img = cv2.imread('41+x3N0H14L._AC_UF1000,1000_QL80_.jpg')
pred_glasses(img) 

('hijab', 0)